In [ ]:
# Penjadwalan Makanan Sugar Glider dengan Algoritma Genetika
# Import library yang diperlukan
import random
import math
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
from datetime import datetime
import numpy as np
import copy

# Load data makanan dan proses indeks kategori
# CATATAN: Asumsi file 'data/data_makanan1.csv' sudah ada dan berisi data yang Anda berikan sebelumnya.
try:
    data_makanan = pd.read_csv('data/data_makanan1.csv')
except FileNotFoundError:
    print("PERINGATAN: File 'data/data_makanan1.csv' tidak ditemukan. Membuat DataFrame dummy berdasarkan output terakhir.")
    # Membuat DataFrame dummy berdasarkan data 204 item terakhir sebagai fallback
    data_from_previous_run = {
        'nama_makanan': ['Jangkrik', 'Ulat Hongkong', 'Apel', 'Wortel', 'Kalsium Bubuk', 'Item Tambahan #1', 'Item Tambahan #51', 'Item Tambahan #101', 'Item Tambahan #151', 'Item Tambahan #197'],
        'jenis': ['protein', 'protein', 'buah', 'sayur', 'supp', 'sayur', 'buah', 'buah', 'buah', 'protein'],
        'protein': [24.0, 30.0, 1.0, 2.0, 0.0, 1.5, 0.5, 5.5, 10.5, 10.0],
        'serat': [1.0, 1.6, 8.0, 12.0, 0.0, 2.0, 0.7, 6.1, 11.1, 0.0],
        'kalsium': [100.0, 200.0, 100.0, 300.0, 100000.0, 20.0, 7.0, 56.0, 106.0, 400.0],
        'fosfor': [400.0, 500.0, 200.0, 200.0, 0.0, 30.0, 10.0, 59.0, 114.0, 450.0],
        'gula': [0.0, 0.0, 25.0, 12.0, 0.0, 2.0, 5.0, 56.0, 106.0, 0.0],
        'kalori': [96.0, 120.0, 52.0, 56.0, 0.0, 10.0, 15.0, 150.0, 250.0, 120.0],
        'berat': [10.0, 10.0, 50.0, 30.0, 0.1, 30.0, 50.0, 50.0, 50.0, 10.0]
    }
    data_makanan = pd.DataFrame(data_from_previous_run)

# Check available columns first
print("Kolom yang tersedia dalam data:")
print(data_makanan.columns.tolist())
print("\nContoh data:")
print(data_makanan.head())

# Create output directory if it doesn't exist
output_dir = './output'
os.makedirs(output_dir, exist_ok=True)

# Process data and create indices
foods = []
protein_idx = []
fruit_idx = []
veg_idx = []
supp_idx = []

# Add kalori and berat columns if missing (estimated values)
if 'kalori' not in data_makanan.columns:
    # Estimate kalori based on protein, fat, carbs (simplified calculation)
    data_makanan['kalori'] = (data_makanan['protein'] * 4 + 
                             data_makanan['gula'] * 4).round(1)

if 'berat' not in data_makanan.columns:
    # Add standard serving weight
    def get_standard_weight(jenis):
        if jenis == 'protein':
            return 10.0  # 10g for protein sources
        elif jenis == 'buah':
            return 50.0  # 50g for fruits
        elif jenis == 'sayur':
            return 30.0  # 30g for vegetables
        elif jenis == 'supp':
            return 0.1    # 0.1g for supplement
        else:
            return 10.0
    
    data_makanan['berat'] = data_makanan['jenis'].apply(get_standard_weight)

for i, row in data_makanan.iterrows():
    try:
        foods.append([
            row['nama_makanan'],      # 0: nama
            row['jenis'],             # 1: kategori
            float(row['protein']) if pd.notna(row['protein']) else 0.0,      # 2: protein
            float(row['serat']) if pd.notna(row['serat']) else 0.0,          # 3: serat
            float(row['kalsium'])/1000 if pd.notna(row['kalsium']) else 0.0, # 4: kalsium (convert mg to g)
            float(row['fosfor'])/1000 if pd.notna(row['fosfor']) else 0.0,   # 5: fosfor (convert mg to g)
            float(row['kalori']) if pd.notna(row['kalori']) else 0.0,         # 6: kalori
            float(row['gula']) if pd.notna(row['gula']) else 0.0,             # 7: gula
            float(row['berat']) if pd.notna(row['berat']) else 10.0           # 8: berat
        ])
        
        jenis = str(row['jenis']).strip().lower()
        
        if jenis == 'protein':
            protein_idx.append(i)
        elif jenis == 'buah':
            fruit_idx.append(i)
        elif jenis == 'sayur':
            veg_idx.append(i)
        elif jenis == 'supp':
            supp_idx.append(i)
    
    except Exception as e:
        print(f"Error processing row {i}: {e}")
        continue

print(f"Data makanan berhasil dimuat: {len(foods)} items")
print(f"Protein: {len(protein_idx)}, Buah: {len(fruit_idx)}, Sayuran: {len(veg_idx)}, Suplemen: {len(supp_idx)}")

# Show category breakdown
print("\nBreakdown kategori:")
print(data_makanan['jenis'].value_counts())

# =======================
# INDIVIDU (7 hari)
# =======================
def random_day():
    return [
        random.choice(protein_idx),
        random.choice(fruit_idx),
        random.choice(veg_idx),
        random.choice([0,1])  # supplement
    ]

def random_individual():
    return [random_day() for _ in range(7)]

# =======================
# HITUNG NUTRISI PER HARI
# =======================
def compute_day_nutrition(day):
    prot, fru, veg, sup = day
    items = [foods[prot], foods[fru], foods[veg]]
    if sup==1 and len(supp_idx) > 0:
        items.append(foods[supp_idx[0]])

    protein = sum(i[2] for i in items)
    fiber    = sum(i[3] for i in items)
    ca       = sum(i[4] for i in items)
    phos     = sum(i[5] for i in items)
    kcal     = sum(i[6] for i in items)
    sugar    = sum(i[7] for i in items)
    mass     = sum(i[8] for i in items)

    return protein, fiber, ca, phos, kcal, sugar, mass

# =======================
# FITNESS FUNCTION - TUNED
# =======================
def fitness(ind):
    total = [compute_day_nutrition(d) for d in ind]
    total_p = sum(x[0] for x in total)
    total_f = sum(x[1] for x in total)
    total_ca = sum(x[2] for x in total)
    total_ph = sum(x[3] for x in total)
    total_kcal = sum(x[4] for x in total)
    total_sugar = sum(x[5] for x in total)
    total_mass = sum(x[6] for x in total)

    if total_kcal == 0 or total_ph == 0:
        return 0

    # Hitung persentase nutrisi
    protein_pct = (4*total_p)/total_kcal*100
    fiber_pct = (total_f/total_mass)*100
    cap_ratio = total_ca/total_ph
    sugar_pct = (4*total_sugar)/total_kcal*100

    # Score untuk setiap komponen nutrisi (DITUNING)
    score_p = max(0, 1 - abs(protein_pct - 30)/20) # Target 30%, batas 20%
    score_f = max(0, 1 - abs(fiber_pct - 35)/20) # Target 35%, batas 20%
    # Gaussian-like score for Ca:P: Target 2:1, std deviasi 0.5
    score_ca = math.exp(-((cap_ratio - 2)**2)/(2 * 0.5**2)) 
    score_s = 1 if sugar_pct <= 10 else max(0, 1-(sugar_pct-10)/15) # Penalti gula di atas 10%
    score_k = 1 if 55 <= total_kcal <= 75 else max(0, 1 - abs(total_kcal-65)/100) # Kisaran Kalori 55-75

    # Variasi makanan
    prot_sources = [d[0] for d in ind]
    variety = len(set(prot_sources))/len(ind)

    # Bobot sederhana (DITUNING)
    final_score = (0.20*score_p + 0.15*score_f + 0.30*score_ca + # Naikkan bobot Ca:P
                     0.10*score_s + 0.15*score_k + 0.10*variety)
    
    return final_score

# =======================
# GENETIC ALGORITHM OPERATORS - TUNED
# =======================

def crossover(parent1, parent2):
    """Single point crossover"""
    point = random.randint(1, 6)
    child1 = parent1[:point] + parent2[point:]
    child2 = parent2[:point] + parent1[point:]
    return child1, child2

def mutate(individual, mutation_rate=0.15): # MUTATION RATE DITINGKATKAN
    """Mutasi sederhana"""
    for day in individual:
        if random.random() < mutation_rate:
            day[0] = random.choice(protein_idx)
        if random.random() < mutation_rate:
            day[1] = random.choice(fruit_idx)
        if random.random() < mutation_rate:
            day[2] = random.choice(veg_idx)
        if random.random() < mutation_rate:
            day[3] = random.choice([0, 1])

def tournament_selection(population, fitnesses, tournament_size=3): # DIGANTI DENGAN TOURNAMENT SELECTION
    """Tournament selection"""
    best_individual = None
    best_fitness = -1
    
    for _ in range(tournament_size):
        idx = random.randint(0, len(population) - 1)
        current_fitness = fitnesses[idx]
        
        if current_fitness > best_fitness:
            best_fitness = current_fitness
            best_individual = population[idx]
    
    return best_individual

# =======================
# GENETIC ALGORITHM - TUNED
# =======================

POP_SIZE = 90
GENERATIONS = 200
MUTATION_RATE = 0.15 # DITUNING
CROSSOVER_RATE = 1.0 # DITUNING

# Track history
best_fitness_history = []
avg_fitness_history = []

# Inisialisasi populasi random
population = [random_individual() for _ in range(POP_SIZE)]

print("Algoritma Genetika Dasar (Tuned untuk Variasi):")
print("="*60)
print(f"Ukuran Populasi: {POP_SIZE}")
print(f"Jumlah Generasi: {GENERATIONS}")
print(f"Mutation Rate: {MUTATION_RATE}")
print(f"Crossover Rate: {CROSSOVER_RATE}")
print("="*60)

for generation in range(GENERATIONS):
    # Evaluasi fitness
    fitnesses = [fitness(ind) for ind in population]
    
    # Track history
    best_fit = max(fitnesses)
    avg_fit = sum(fitnesses) / len(fitnesses)
    best_fitness_history.append(best_fit)
    avg_fitness_history.append(avg_fit)
    
    # Print progress
    if generation % 10 == 0:
        print(f"Gen {generation:3d}: Best={best_fit:.4f}, Avg={avg_fit:.4f}")
    
    # Buat populasi baru
    new_population = []

    # Elitism: simpan 2 individu terbaik
    sorted_pop = sorted(zip(population, fitnesses), key=lambda x: x[1], reverse=True)
    # **PERBAIKAN: Menggunakan copy.deepcopy untuk memastikan salinan mendalam**
    new_population.extend([copy.deepcopy(sorted_pop[0][0]), copy.deepcopy(sorted_pop[1][0])])
    # ...
    
    # Buat offspring sampai populasi penuh
    while len(new_population) < POP_SIZE:
        # Selection (Menggunakan Tournament)
        parent1 = tournament_selection(population, fitnesses)
        parent2 = tournament_selection(population, fitnesses)
        
        # Crossover
        if random.random() < CROSSOVER_RATE:
            child1, child2 = crossover(parent1, parent2)
        else:
            child1, child2 = parent1[:], parent2[:]
        
        # Mutation
        mutate(child1, MUTATION_RATE)
        mutate(child2, MUTATION_RATE)
        
        new_population.append(child1)
        if len(new_population) < POP_SIZE:
            new_population.append(child2)
    
    population = new_population

print(f"\nGenerasi {GENERATIONS}: Best={max([fitness(ind) for ind in population]):.4f}")
print("Evolusi selesai!")
# =======================
# ANALISIS HASIL TERBAIK
# =======================

best = max(population, key=fitness)
days = ["Senin","Selasa","Rabu","Kamis","Jumat","Sabtu","Minggu"]

# Create detailed schedule
result = []
nutrition_per_day = []

for i, day in enumerate(best):
    prot, fru, veg, sup = day
    day_nutrition = compute_day_nutrition(day)
    
    result.append([
        days[i],
        foods[prot][0],
        foods[fru][0],
        foods[veg][0],
        "Ya" if sup==1 else "Tidak"
    ])
    
    nutrition_per_day.append({
        'Hari': days[i],
        'Protein': day_nutrition[0],
        'Serat': day_nutrition[1],
        'Kalsium': day_nutrition[2],
        'Fosfor': day_nutrition[3],
        'Kalori': day_nutrition[4],
        'Gula': day_nutrition[5],
        'Massa': day_nutrition[6]
    })

# Create DataFrames
df_schedule = pd.DataFrame(result, columns=["Hari","Protein","Buah","Sayur","Suplemen Kalsium"])
df_nutrition = pd.DataFrame(nutrition_per_day)

# Calculate totals and percentages
total_nutrition = df_nutrition.sum(numeric_only=True)
total_kcal = total_nutrition['Kalori']
total_mass = total_nutrition['Massa']

protein_pct = (4 * total_nutrition['Protein']) / total_kcal * 100
fiber_pct = (total_nutrition['Serat'] / total_mass) * 100
ca_phos_ratio = total_nutrition['Kalsium'] / total_nutrition['Fosfor']
sugar_pct = (4 * total_nutrition['Gula']) / total_kcal * 100

print("JADWAL MAKANAN OPTIMAL UNTUK SUGAR GLIDER")
print("="*50)
display(df_schedule)
print(f"\nFitness Score: {fitness(best):.4f}")
# =======================
# VISUALISASI DAN OUTPUT - GRAFIK TERPISAH
# =======================

# Set style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

# Create output directories for plots
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
plots_dir = f'{output_dir}/plots_{timestamp}_TUNED'
os.makedirs(plots_dir, exist_ok=True)

print("\nMenyimpan grafik terpisah (Versi Tuned):")
print("="*40)

# 1. Evolution of fitness (Grafik ini seharusnya menunjukkan konvergensi yang lebih lambat)
plt.figure(figsize=(12, 8))
generations = range(len(best_fitness_history))
plt.plot(generations, best_fitness_history, 'b-', linewidth=2, label='Best Fitness')
plt.plot(generations, avg_fitness_history, 'r--', linewidth=2, label='Average Fitness')
plt.title('Evolusi Fitness Algoritma Genetika (Tuned)', fontsize=16, fontweight='bold')
plt.xlabel('Generasi')
plt.ylabel('Fitness Score')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
fitness_plot = f'{plots_dir}/01_evolusi_fitness_tuned.png'
plt.savefig(fitness_plot, dpi=300, bbox_inches='tight')
plt.close()
print(f"✓ Evolusi fitness: {fitness_plot}")

# 2. Nutrition per day (Calories)
plt.figure(figsize=(12, 8))
x_pos = np.arange(len(days))
bars = plt.bar(x_pos, df_nutrition['Kalori'], color='orange', alpha=0.7)
plt.title('Kalori per Hari (Tuned Range)', fontsize=16, fontweight='bold')
plt.xlabel('Hari')
plt.ylabel('Kalori (kcal)')
plt.xticks(x_pos, days, rotation=45)
plt.axhline(y=65, color='red', linestyle='--', label='Target (65 kcal)')
plt.legend()
for i, bar in enumerate(bars):
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2., height + 0.5, f'{height:.1f}', 
             ha='center', va='bottom', fontsize=10)
plt.tight_layout()
calories_plot = f'{plots_dir}/02_kalori_per_hari.png'
plt.savefig(calories_plot, dpi=300, bbox_inches='tight')
plt.close()
print(f"✓ Kalori per hari: {calories_plot}")

# 3. Protein sources variety
plt.figure(figsize=(10, 8))
protein_sources = [foods[day[0]][0] for day in best]
protein_counts = pd.Series(protein_sources).value_counts()
wedges, texts, autotexts = plt.pie(protein_counts.values, labels=protein_counts.index, 
                                   autopct='%1.1f%%', startangle=90)
plt.title('Variasi Sumber Protein (Tuned)', fontsize=16, fontweight='bold')
plt.tight_layout()
protein_variety_plot = f'{plots_dir}/03_variasi_protein.png'
plt.savefig(protein_variety_plot, dpi=300, bbox_inches='tight')
plt.close()
print(f"✓ Variasi protein: {protein_variety_plot}")

# 4. Nutrient composition
plt.figure(figsize=(12, 8))
nutrients = ['Protein', 'Serat', 'Kalsium', 'Fosfor', 'Gula']
values = [total_nutrition[n] for n in nutrients]
bars = plt.bar(nutrients, values, color=['red', 'green', 'blue', 'purple', 'orange'], alpha=0.7)
plt.title('Komposisi Nutrisi Total (7 hari)', fontsize=16, fontweight='bold')
plt.ylabel('Jumlah (gram)')
plt.xticks(rotation=45)
for i, bar in enumerate(bars):
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2., height + 0.1, f'{height:.2f}g', 
             ha='center', va='bottom', fontsize=10)
plt.tight_layout()
nutrients_plot = f'{plots_dir}/04_komposisi_nutrisi.png'
plt.savefig(nutrients_plot, dpi=300, bbox_inches='tight')
plt.close()
print(f"✓ Komposisi nutrisi: {nutrients_plot}")

# 5. Ca:P ratio per day
plt.figure(figsize=(12, 8))
ca_p_ratios = [row['Kalsium']/row['Fosfor'] if row['Fosfor'] > 0 else 0 for _, row in df_nutrition.iterrows()]
bars = plt.bar(days, ca_p_ratios, color='lightblue', alpha=0.7)
plt.title('Rasio Ca:P per Hari', fontsize=16, fontweight='bold')
plt.ylabel('Rasio Ca:P')
plt.axhline(y=2.0, color='red', linestyle='--', label='Target (2:1)')
plt.xticks(rotation=45)
plt.legend()
for i, bar in enumerate(bars):
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2., height + 0.05, f'{height:.2f}', 
             ha='center', va='bottom', fontsize=9)
plt.tight_layout()
ratio_plot = f'{plots_dir}/05_rasio_ca_p.png'
plt.savefig(ratio_plot, dpi=300, bbox_inches='tight')
plt.close()
print(f"✓ Rasio Ca:P: {ratio_plot}")

# 6. Nutritional percentages
plt.figure(figsize=(12, 8))
categories = ['Protein %', 'Serat %', 'Gula %']
percentages = [protein_pct, fiber_pct, sugar_pct]
targets = [30, 35, 10]
colors = ['red', 'green', 'orange']

x_pos = np.arange(len(categories))
bars1 = plt.bar(x_pos - 0.2, percentages, 0.4, label='Aktual', color=colors, alpha=0.7)
bars2 = plt.bar(x_pos + 0.2, targets, 0.4, label='Target', color=colors, alpha=0.3)
plt.title('Persentase Nutrisi vs Target (Tuned)', fontsize=16, fontweight='bold')
plt.ylabel('Persentase (%)')
plt.xticks(x_pos, categories)
plt.legend()
plt.tight_layout()
percentage_plot = f'{plots_dir}/06_persentase_nutrisi.png'
plt.savefig(percentage_plot, dpi=300, bbox_inches='tight')
plt.close()
print(f"✓ Persentase nutrisi: {percentage_plot}")

# 7. Weekly nutrition trend
plt.figure(figsize=(12, 8))
plt.plot(days, df_nutrition['Protein'], 'o-', label='Protein', linewidth=2, markersize=8)
plt.plot(days, df_nutrition['Serat'], 's-', label='Serat', linewidth=2, markersize=8)
plt.plot(days, df_nutrition['Gula'], '^-', label='Gula', linewidth=2, markersize=8)
plt.title('Tren Nutrisi Mingguan', fontsize=16, fontweight='bold')
plt.ylabel('Gram')
plt.xticks(rotation=45)
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
trend_plot = f'{plots_dir}/07_tren_nutrisi_mingguan.png'
plt.savefig(trend_plot, dpi=300, bbox_inches='tight')
plt.close()
print(f"✓ Tren nutrisi mingguan: {trend_plot}")

# 8. Supplement usage
plt.figure(figsize=(10, 8))
supplement_days = [day[3] for day in best]
supp_count = sum(supplement_days)
no_supp_count = 7 - supp_count
plt.pie([supp_count, no_supp_count], labels=['Dengan Suplemen', 'Tanpa Suplemen'], 
        autopct='%1.1f%%', colors=['lightgreen', 'lightcoral'])
plt.title('Penggunaan Suplemen Kalsium', fontsize=16, fontweight='bold')
plt.tight_layout()
supplement_plot = f'{plots_dir}/08_penggunaan_suplemen.png'
plt.savefig(supplement_plot, dpi=300, bbox_inches='tight')
plt.close()
print(f"✓ Penggunaan suplemen: {supplement_plot}")

# 9. Fitness components breakdown
plt.figure(figsize=(12, 8))
# Calculate individual fitness components (re-calculating scores with new parameters)
total = [compute_day_nutrition(d) for d in best]
total_p = sum(x[0] for x in total)
total_f = sum(x[1] for x in total)
total_ca = sum(x[2] for x in total)
total_ph = sum(x[3] for x in total)
total_kcal = sum(x[4] for x in total)
total_sugar = sum(x[5] for x in total)
total_mass = sum(x[6] for x in total)

protein_pct_calc = (4*total_p)/total_kcal*100
fiber_pct_calc = (total_f/total_mass)*100
cap_ratio_calc = total_ca/total_ph
sugar_pct_calc = (4*total_sugar)/total_kcal*100

score_p = max(0, 1 - abs(protein_pct_calc - 30)/20) # Sesuai tuning fitness
score_f = max(0, 1 - abs(fiber_pct_calc - 35)/20)
score_ca = math.exp(-((cap_ratio_calc-2)**2)/(2*0.5**2))
score_s = 1 if sugar_pct_calc <= 10 else max(0, 1-(sugar_pct_calc-10)/15)
score_k = 1 if 55 <= total_kcal <= 75 else max(0, 1 - abs(total_kcal-65)/100)

prot_sources = [d[0] for d in best]
variety = len(set(prot_sources))/len(best)
score_v = variety

components = ['Protein', 'Serat', 'Ca:P', 'Gula', 'Kalori', 'Variasi']
scores = [score_p, score_f, score_ca, score_s, score_k, score_v]
bars = plt.bar(components, scores, color='skyblue', alpha=0.7)
plt.title('Breakdown Komponen Fitness (Tuned)', fontsize=16, fontweight='bold')
plt.ylabel('Score (0-1)')
plt.xticks(rotation=45)
plt.ylim(0, 1.1)
for i, bar in enumerate(bars):
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2., height + 0.02, f'{height:.3f}', 
             ha='center', va='bottom', fontsize=10)
plt.tight_layout()
fitness_breakdown_plot = f'{plots_dir}/09_breakdown_fitness.png'
plt.savefig(fitness_breakdown_plot, dpi=300, bbox_inches='tight')
plt.close()
print(f"✓ Breakdown fitness: {fitness_breakdown_plot}")

# 10. Daily nutrition comparison (Bonus chart)
plt.figure(figsize=(14, 8))
width = 0.15
x = np.arange(len(days))

plt.bar(x - 2*width, df_nutrition['Protein'], width, label='Protein (g)', alpha=0.8)
plt.bar(x - width, df_nutrition['Serat'], width, label='Serat (g)', alpha=0.8)
plt.bar(x, df_nutrition['Kalsium']*1000, width, label='Kalsium (mg)', alpha=0.8)
plt.bar(x + width, df_nutrition['Fosfor']*1000, width, label='Fosfor (mg)', alpha=0.8)
plt.bar(x + 2*width, df_nutrition['Gula'], width, label='Gula (g)', alpha=0.8)

plt.title('Perbandingan Nutrisi Harian', fontsize=16, fontweight='bold')
plt.xlabel('Hari')
plt.ylabel('Jumlah')
plt.xticks(x, days, rotation=45)
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
daily_comparison_plot = f'{plots_dir}/10_perbandingan_nutrisi_harian.png'
plt.savefig(daily_comparison_plot, dpi=300, bbox_inches='tight')
plt.close()
print(f"✓ Perbandingan nutrisi harian: {daily_comparison_plot}")

print(f"\n✅ Semua 10 grafik berhasil disimpan di folder: {plots_dir}")
print(f"📁 Total file grafik: 10 file PNG")
# =======================
# SAVE DETAILED REPORTS
# =======================

# Create summary report
summary_data = {
    'Metrik': [
        'Total Kalori (7 hari)',
        'Total Protein (g)',
        'Total Serat (g)', 
        'Total Kalsium (g)',
        'Total Fosfor (g)',
        'Total Gula (g)',
        'Persentase Protein (%)',
        'Persentase Serat (%)',
        'Rasio Ca:P',
        'Persentase Gula (%)',
        'Fitness Score',
        'Variasi Protein'
    ],
    'Nilai': [
        f"{total_nutrition['Kalori']:.1f} kcal",
        f"{total_nutrition['Protein']:.2f} g",
        f"{total_nutrition['Serat']:.2f} g",
        f"{total_nutrition['Kalsium']:.2f} g",
        f"{total_nutrition['Fosfor']:.2f} g",
        f"{total_nutrition['Gula']:.2f} g",
        f"{protein_pct:.1f}%",
        f"{fiber_pct:.1f}%",
        f"{ca_phos_ratio:.2f}:1",
        f"{sugar_pct:.1f}%",
        f"{fitness(best):.4f}",
        f"{len(set([foods[day[0]][0] for day in best]))}/7 jenis"
    ],
    'Target/Status': [
        "55-75 kcal (Disesuaikan)",
        "Dekat 30% dari kalori",
        "Dekat 35% dari massa",
        "Rasio 2:1 dengan P",
        "Seimbang dengan Ca",
        "≤10% dari kalori",
        "30% (target)",
        "35% (target)",
        "2:1 (optimal)",
        "≤10% (target)",
        "Maksimal 1.0",
        "Semakin bervariasi semakin baik"
    ]
}

df_summary = pd.DataFrame(summary_data)

# Save Excel file with multiple sheets
excel_filename = f'{output_dir}/jadwal_makanan_sugar_glider_{timestamp}_TUNED.xlsx'
with pd.ExcelWriter(excel_filename, engine='openpyxl') as writer:
    df_schedule.to_excel(writer, sheet_name='Jadwal Makanan', index=False)
    df_nutrition.to_excel(writer, sheet_name='Nutrisi Harian', index=False)
    df_summary.to_excel(writer, sheet_name='Ringkasan Nutrisi', index=False)
    
    # Fitness evolution data
    df_fitness = pd.DataFrame({
        'Generasi': range(len(best_fitness_history)),
        'Best Fitness': best_fitness_history,
        'Average Fitness': avg_fitness_history
    })
    df_fitness.to_excel(writer, sheet_name='Evolusi Fitness', index=False)

print(f"Laporan Excel disimpan: {excel_filename}")

# Save CSV files
csv_schedule = f'{output_dir}/jadwal_makanan_{timestamp}_TUNED.csv'
csv_nutrition = f'{output_dir}/nutrisi_harian_{timestamp}_TUNED.csv'
csv_summary = f'{output_dir}/ringkasan_nutrisi_{timestamp}_TUNED.csv'

df_schedule.to_csv(csv_schedule, index=False)
df_nutrition.to_csv(csv_nutrition, index=False)
df_summary.to_csv(csv_summary, index=False)

print(f"File CSV disimpan:")
print(f"- {csv_schedule}")
print(f"- {csv_nutrition}")
print(f"- {csv_summary}")
# =======================
# DISPLAY FINAL RESULTS
# =======================

print("\n" + "="*60)
print("RINGKASAN HASIL OPTIMASI (TUNED)")
print("="*60)
display(df_summary)

print("\n" + "="*60)
print("NUTRISI HARIAN DETAIL")
print("="*60)
display(df_nutrition.round(2))

print(f"\nSemua output telah disimpan di folder: {plots_dir}")
print(f"Timestamp: {timestamp}")

Kolom yang tersedia dalam data:
['nama_makanan', 'jenis', 'protein', 'serat', 'kalsium', 'fosfor', 'gula']

Contoh data:
    nama_makanan    jenis  protein  serat  kalsium  fosfor  gula
0       Jangkrik  protein    24.00    1.0    100.0   400.0   0.0
1  Ulat Hongkong  protein    30.00    1.6    200.0   500.0   0.0
2     Ayam Rebus  protein    30.00    0.0     60.0   400.0   0.0
3    Telur Rebus  protein    25.00    0.0    200.0   900.0   0.0
4       HPW Diet  protein    33.33   10.0   1333.3   666.7  10.0
Data makanan berhasil dimuat: 18 items
Protein: 5, Buah: 6, Sayuran: 6, Suplemen: 1

Breakdown kategori:
jenis
buah       6
sayur      6
protein    5
supp       1
Name: count, dtype: int64
Algoritma Genetika Dasar (Tuned untuk Variasi):
Ukuran Populasi: 90
Jumlah Generasi: 200
Mutation Rate: 0.15
Crossover Rate: 1.0
Gen   0: Best=0.1762, Avg=0.1101
Gen  10: Best=0.2198, Avg=0.0971
Gen  20: Best=0.2198, Avg=0.1178
Gen  30: Best=0.2366, Avg=0.1009
Gen  40: Best=0.2780, Avg=0.1228
Gen  5

,Hari,Protein,Buah,Sayur,Suplemen Kalsium
0,Senin,HPW Diet,Pepaya,Selada,Tidak
1,Selasa,Jangkrik,Pepaya,Selada,Tidak
2,Rabu,HPW Diet,Pir,Selada,Tidak
3,Kamis,HPW Diet,Pisang,Bayam,Tidak
4,Jumat,HPW Diet,Anggur,Buncis,Tidak
5,Sabtu,HPW Diet,Apel,Bayam,Tidak
6,Minggu,HPW Diet,Pir,Bayam,Tidak



Fitness Score: 0.3883

Menyimpan grafik terpisah (Versi Tuned):
✓ Evolusi fitness: ./output/plots_20251105_225905_TUNED/01_evolusi_fitness_tuned.png
✓ Kalori per hari: ./output/plots_20251105_225905_TUNED/02_kalori_per_hari.png
✓ Variasi protein: ./output/plots_20251105_225905_TUNED/03_variasi_protein.png
✓ Komposisi nutrisi: ./output/plots_20251105_225905_TUNED/04_komposisi_nutrisi.png
✓ Rasio Ca:P: ./output/plots_20251105_225905_TUNED/05_rasio_ca_p.png
✓ Persentase nutrisi: ./output/plots_20251105_225905_TUNED/06_persentase_nutrisi.png
✓ Tren nutrisi mingguan: ./output/plots_20251105_225905_TUNED/07_tren_nutrisi_mingguan.png
✓ Penggunaan suplemen: ./output/plots_20251105_225905_TUNED/08_penggunaan_suplemen.png
✓ Breakdown fitness: ./output/plots_20251105_225905_TUNED/09_breakdown_fitness.png
✓ Perbandingan nutrisi harian: ./output/plots_20251105_225905_TUNED/10_perbandingan_nutrisi_harian.png

✅ Semua 10 grafik berhasil disimpan di folder: ./output/plots_20251105_225905_TUNED
📁 Tota

,Metrik,Nilai,Target/Status
0,Total Kalori (7 hari),2097.4 kcal,55-75 kcal (Disesuaikan)
1,Total Protein (g),255.38 g,Dekat 30% dari kalori
2,Total Serat (g),196.80 g,Dekat 35% dari massa
3,Total Kalsium (g),11.80 g,Rasio 2:1 dengan P
4,Total Fosfor (g),7.28 g,Seimbang dengan Ca
5,Total Gula (g),269.00 g,≤10% dari kalori
6,Persentase Protein (%),48.7%,30% (target)
7,Persentase Serat (%),31.2%,35% (target)
8,Rasio Ca:P,1.62:1,2:1 (optimal)
9,Persentase Gula (%),51.3%,≤10% (target)



NUTRISI HARIAN DETAIL


,Hari,Protein,Serat,Kalsium,Fosfor,Kalori,Gula,Massa
0,Senin,36.53,29.0,1.79,1.03,274.1,32.0,90.0
1,Selasa,27.20,20.0,0.56,0.76,196.8,22.0,90.0
2,Rabu,35.93,28.4,1.73,1.01,291.7,37.0,90.0
3,Kamis,39.33,29.0,2.03,1.27,373.3,54.0,90.0
4,Jumat,39.73,26.0,1.81,1.07,342.9,46.0,90.0
5,Sabtu,38.33,32.0,1.93,1.07,309.3,39.0,90.0
6,Minggu,38.33,32.4,1.93,1.09,309.3,39.0,90.0



Semua output telah disimpan di folder: ./output/plots_20251105_225905_TUNED
Timestamp: 20251105_225905
